In [1]:
import numpy, scipy, matplotlib.pyplot as plt, sklearn, librosa, mir_eval, IPython.display, urllib

[&larr; Back to Index](index.html)

# Unsupervised Instrument Classification Using K-Means 

This lab is loosely based on [Lab 3](https://ccrma.stanford.edu/workshops/mir2010/Lab3_2010.pdf) (2010).

## Read Audio

Retrieve an audio file, load it into an array, and listen to it.

In [2]:
librosa.load?

Signature:
librosa.load(
    path: 'Union[str, int, os.PathLike[Any], sf.SoundFile, audioread.AudioFile, BinaryIO]',
    *,
    sr: 'Optional[float]' = 22050,
    mono: 'bool' = True,
    offset: 'float' = 0.0,
    duration: 'Optional[float]' = None,
    dtype: 'DTypeLike' = <class 'numpy.float32'>,
    res_type: 'str' = 'soxr_hq',
) -> 'Tuple[np.ndarray, float]'
Docstring:
Load an audio file as a floating point time series.

Audio will be automatically resampled to the given rate
(default ``sr=22050``).

To preserve the native sampling rate of the file, use ``sr=None``.

Parameters
----------
path : string, int, pathlib.Path, soundfile.SoundFile, audioread object, or file-like object
    path to the input file.

    Any codec supported by `soundfile` or `audioread` will work.

    Any string file paths, or any object implementing Python's
    file interface (e.g. `pathlib.Path`) are supported as `path`.

    If the codec is supported by `soundfile`, then `path` can also be
    an open

In [3]:
IPython.display.Audio?

Init signature:
IPython.display.Audio(
    data=None,
    filename=None,
    url=None,
    embed=None,
    rate=None,
    autoplay=False,
    normalize=True,
    *,
    element_id=None,
)
Docstring:     
Create an audio object.

When this object is returned by an input cell or passed to the
display function, it will result in Audio controls being displayed
in the frontend (only works in the notebook).

Parameters
----------
data : numpy array, list, unicode, str or bytes
    Can be one of

      * Numpy 1d array containing the desired waveform (mono)
      * Numpy 2d array containing waveforms for each channel.
        Shape=(NCHAN, NSAMPLES). For the standard channel order, see
        http://msdn.microsoft.com/en-us/library/windows/hardware/dn653308(v=vs.85).aspx
      * List of float or integer representing the waveform (mono)
      * String containing the filename
      * Bytestring containing raw PCM data or
      * URL pointing to a file on the web.

    If the array option is us

## Detect Onsets

Detect onsets in the audio signal:

In [4]:
librosa.onset.onset_detect?

Signature:
librosa.onset.onset_detect(
    *,
    y: Optional[numpy.ndarray] = None,
    sr: float = 22050,
    onset_envelope: Optional[numpy.ndarray] = None,
    hop_length: int = 512,
    backtrack: bool = False,
    energy: Optional[numpy.ndarray] = None,
    units: str = 'frames',
    normalize: bool = True,
    **kwargs: Any,
) -> numpy.ndarray
Docstring:
Locate note onset events by picking peaks in an onset strength envelope.

The `peak_pick` parameters were chosen by large-scale hyper-parameter
optimization over the dataset provided by [#]_.

.. [#] https://github.com/CPJKU/onset_db

Parameters
----------
y : np.ndarray [shape=(n,)]
    audio time series, must be monophonic

sr : number > 0 [scalar]
    sampling rate of ``y``

onset_envelope : np.ndarray [shape=(m,)]
    (optional) pre-computed onset strength envelope

hop_length : int > 0 [scalar]
    hop length (in samples)

units : {'frames', 'samples', 'time'}
    The units to encode detected onset events in.
    By default

Convert the onsets from units of frames to seconds (and samples):

In [5]:
librosa.frames_to_time?

Signature:
librosa.frames_to_time(
    frames: '_ScalarOrSequence[_IntLike_co]',
    *,
    sr: 'float' = 22050,
    hop_length: 'int' = 512,
    n_fft: 'Optional[int]' = None,
) -> 'Union[np.floating[Any], np.ndarray]'
Docstring:
Convert frame counts to time (seconds).

Parameters
----------
frames : np.ndarray [shape=(n,)]
    frame index or vector of frame indices
sr : number > 0 [scalar]
    audio sampling rate
hop_length : int > 0 [scalar]
    number of samples between successive frames
n_fft : None or int > 0 [scalar]
    Optional: length of the FFT window.
    If given, time conversion will include an offset of ``n_fft // 2``
    to counteract windowing effects when using a non-centered STFT.

Returns
-------
times : np.ndarray [shape=(n,)]
    time (in seconds) of each given frame number::

        times[i] = frames[i] * hop_length / sr

See Also
--------
time_to_frames : convert time values to frame indices
frames_to_samples : convert frame indices to sample indices

Examples


In [6]:
librosa.frames_to_samples?

Signature:
librosa.frames_to_samples(
    frames: '_ScalarOrSequence[_IntLike_co]',
    *,
    hop_length: 'int' = 512,
    n_fft: 'Optional[int]' = None,
) -> 'Union[np.integer[Any], np.ndarray]'
Docstring:
Convert frame indices to audio sample indices.

Parameters
----------
frames : number or np.ndarray [shape=(n,)]
    frame index or vector of frame indices
hop_length : int > 0 [scalar]
    number of samples between successive frames
n_fft : None or int > 0 [scalar]
    Optional: length of the FFT window.
    If given, time conversion will include an offset of ``n_fft // 2``
    to counteract windowing effects when using a non-centered STFT.

Returns
-------
times : number or np.ndarray
    time (in samples) of each given frame number::

        times[i] = frames[i] * hop_length

See Also
--------
frames_to_time : convert frame indices to time values
samples_to_frames : convert sample indices to frame indices

Examples
--------
>>> y, sr = librosa.load(librosa.ex('choice'))
>>> tem

Listen to detected onsets:

In [7]:
mir_eval.sonify.clicks?

Signature: mir_eval.sonify.clicks(times, fs, click=None, length=None)
Docstring:
Returns a signal with the signal 'click' placed at each specified time

Parameters
----------
times : np.ndarray
    times to place clicks, in seconds
fs : int
    desired sampling rate of the output signal
click : np.ndarray
    click signal, defaults to a 1 kHz blip
length : int
    desired number of samples in the output signal,
    defaults to ``times.max()*fs + click.shape[0] + 1``

Returns
-------
click_signal : np.ndarray
    Synthesized click signal
File:      ~/Github/iranroman/musicinformationretrieval.com/venv/lib/python3.11/site-packages/mir_eval/sonify.py
Type:      function

In [8]:
IPython.display.Audio?

Init signature:
IPython.display.Audio(
    data=None,
    filename=None,
    url=None,
    embed=None,
    rate=None,
    autoplay=False,
    normalize=True,
    *,
    element_id=None,
)
Docstring:     
Create an audio object.

When this object is returned by an input cell or passed to the
display function, it will result in Audio controls being displayed
in the frontend (only works in the notebook).

Parameters
----------
data : numpy array, list, unicode, str or bytes
    Can be one of

      * Numpy 1d array containing the desired waveform (mono)
      * Numpy 2d array containing waveforms for each channel.
        Shape=(NCHAN, NSAMPLES). For the standard channel order, see
        http://msdn.microsoft.com/en-us/library/windows/hardware/dn653308(v=vs.85).aspx
      * List of float or integer representing the waveform (mono)
      * String containing the filename
      * Bytestring containing raw PCM data or
      * URL pointing to a file on the web.

    If the array option is us

## Extract Features

Extract a set of features from the audio at each onset. Use any of the features we have learned so far: zero crossing rate, spectral moments, MFCCs, chroma, etc. For more, see the [librosa API reference](http://bmcfee.github.io/librosa/index.html).

First, define which features to extract:

In [9]:
def extract_features(x, fs):
    feature_1 = librosa.zero_crossings(x).sum() # placeholder
    feature_2 = 0 # placeholder
    return [feature_1, feature_2]

For each onset, extract a feature vector from the signal:

In [10]:
# Assumptions:
# x: input audio signal
# fs: sampling frequency
# onset_samples: onsets in units of samples
fs = 44100
frame_sz = fs*0.100
#features = numpy.array([extract_features(x[i:i+frame_sz], fs) for i in onset_samples])

## Scale Features

Use `sklearn.preprocessing.MinMaxScaler` to scale your features to be within `[-1, 1]`.

In [11]:
sklearn.preprocessing.MinMaxScaler?

Object `sklearn.preprocessing.MinMaxScaler` not found.


In [12]:
sklearn.preprocessing.MinMaxScaler.fit_transform?

Object `sklearn.preprocessing.MinMaxScaler.fit_transform` not found.


## Plot Features

Use `scatter` to plot features on a 2-D plane. (Choose two features at a time.)

In [13]:
plt.scatter?

Signature:
plt.scatter(
    x,
    y,
    s=None,
    c=None,
    marker=None,
    cmap=None,
    norm=None,
    vmin=None,
    vmax=None,
    alpha=None,
    linewidths=None,
    *,
    edgecolors=None,
    plotnonfinite=False,
    data=None,
    **kwargs,
)
Docstring:
A scatter plot of *y* vs. *x* with varying marker size and/or color.

Parameters
----------
x, y : float or array-like, shape (n, )
    The data positions.

s : float or array-like, shape (n, ), optional
    The marker size in points**2 (typographic points are 1/72 in.).
    Default is ``rcParams['lines.markersize'] ** 2``.

c : array-like or list of colors or color, optional
    The marker colors. Possible values:

    - A scalar or sequence of n numbers to be mapped to colors using
      *cmap* and *norm*.
    - A 2D array in which the rows are RGB or RGBA.
    - A sequence of colors of length n.
    - A single color format string.

    Note that *c* should not be a single numeric RGB or RGBA sequence
    because that

## Cluster Using K-Means

Use `KMeans` to cluster your features and compute labels.

In [14]:
sklearn.cluster.KMeans?

Object `sklearn.cluster.KMeans` not found.


In [15]:
sklearn.cluster.KMeans.fit_predict?

Object `sklearn.cluster.KMeans.fit_predict` not found.


## Plot Features by Class Label

Use `scatter`, but this time choose a different marker color (or type) for each class.

In [16]:
plt.scatter?

Signature:
plt.scatter(
    x,
    y,
    s=None,
    c=None,
    marker=None,
    cmap=None,
    norm=None,
    vmin=None,
    vmax=None,
    alpha=None,
    linewidths=None,
    *,
    edgecolors=None,
    plotnonfinite=False,
    data=None,
    **kwargs,
)
Docstring:
A scatter plot of *y* vs. *x* with varying marker size and/or color.

Parameters
----------
x, y : float or array-like, shape (n, )
    The data positions.

s : float or array-like, shape (n, ), optional
    The marker size in points**2 (typographic points are 1/72 in.).
    Default is ``rcParams['lines.markersize'] ** 2``.

c : array-like or list of colors or color, optional
    The marker colors. Possible values:

    - A scalar or sequence of n numbers to be mapped to colors using
      *cmap* and *norm*.
    - A 2D array in which the rows are RGB or RGBA.
    - A sequence of colors of length n.
    - A single color format string.

    Note that *c* should not be a single numeric RGB or RGBA sequence
    because that

## Listen to Click Track

Create a beep for each onset within a class:

In [17]:
#beeps = mir_eval.sonify.clicks(onset_times[labels==0], fs, length=len(x))

In [18]:
IPython.display.Audio?

Init signature:
IPython.display.Audio(
    data=None,
    filename=None,
    url=None,
    embed=None,
    rate=None,
    autoplay=False,
    normalize=True,
    *,
    element_id=None,
)
Docstring:     
Create an audio object.

When this object is returned by an input cell or passed to the
display function, it will result in Audio controls being displayed
in the frontend (only works in the notebook).

Parameters
----------
data : numpy array, list, unicode, str or bytes
    Can be one of

      * Numpy 1d array containing the desired waveform (mono)
      * Numpy 2d array containing waveforms for each channel.
        Shape=(NCHAN, NSAMPLES). For the standard channel order, see
        http://msdn.microsoft.com/en-us/library/windows/hardware/dn653308(v=vs.85).aspx
      * List of float or integer representing the waveform (mono)
      * String containing the filename
      * Bytestring containing raw PCM data or
      * URL pointing to a file on the web.

    If the array option is us

## Listen to Clustered Frames

Use the `concatenated_segments` function from the [feature sonification exercise](feature_sonification.html) to concatenate frames from the same cluster into one signal. Then listen to the signal. 

In [19]:
def concatenate_segments(segments, fs=44100, pad_time=0.300):
    padded_segments = [numpy.concatenate([segment, numpy.zeros(int(pad_time*fs))]) for segment in segments]
    return numpy.concatenate(padded_segments)
##concatenated_signal = concatenate_segments(segments, fs)

Compare across separate classes. What do you hear?

## For Further Exploration

Use a different number of clusters in `KMeans`.

Use a different initialization method in `KMeans`.

Use different features. Compare tonal features against timbral features.

In [20]:
librosa.feature?

Type:        module
String form: <module 'librosa.feature' from '/Users/gijsm/Github/iranroman/musicinformationretrieval.com/venv/lib/python3.11/site-packages/librosa/feature/__init__.py'>
File:        ~/Github/iranroman/musicinformationretrieval.com/venv/lib/python3.11/site-packages/librosa/feature/__init__.py
Docstring:  
Feature extraction

Spectral features
-----------------

.. autosummary::
    :toctree: generated/

    chroma_stft
    chroma_cqt
    chroma_cens
    chroma_vqt
    melspectrogram
    mfcc
    rms
    spectral_centroid
    spectral_bandwidth
    spectral_contrast
    spectral_flatness
    spectral_rolloff
    poly_features
    tonnetz
    zero_crossing_rate

Rhythm features
---------------
.. autosummary::
    :toctree: generated/

    tempo
    tempogram
    fourier_tempogram
    tempogram_ratio

Feature manipulation
--------------------

.. autosummary::
    :toctree: generated/

    delta
    stack_memory


Feature inversion
-----------------

.. autosummary::
 

Use different audio files.

In [21]:
#filename = '1_bar_funk_groove.mp3'
#filename = '58bpm.wav'
#filename = '125_bounce.wav'
#filename = 'prelude_cmaj_10s.wav'

[&larr; Back to Index](index.html)